In [ ]:
import shutil
import os
import re
from datetime import datetime
import tarfile
import zipfile

from torchvision import datasets
import torch

def log_message(message, level="info"):
    """
    Log a message, with support for info, warning, and error levels.
    If the level is "error", this function raises a ValueError with the message.

    Parameters:
        message (str): The message to log.
        level (str): The level of the message ("info", "warning", or "error").
    """
    if level == "info":
        print(f"[INFO] {message}")
    elif level == "warning":
        print(f"[WARNING] {message}")
    elif level == "error":
        print(f"[ERROR] {message}")
        raise ValueError(message)  # Raise an error if level is "error"

def sanitize_string(s):
    """
    Convert a string to lowercase, strip special characters, and replace spaces with underscores.

    Parameters:
        s (str): The input string to sanitize.

    Returns:
        str: The sanitized string.
    """
    s = s.lower()
    s = s.replace(' ', '_')
    s = re.sub(r'[^a-z0-9_]', '', s)
    return s

def ensure_directory_exists(dir_path):
    """
    Ensure the specified directory exists; create it if it does not, including required subdirectories.
    Set permissions to 777 for the main directory and its contents recursively.

    Parameters:
        dir_path (str): The path of the directory to ensure exists.

    Returns:
        str: The absolute path of the directory.
    """
    os.makedirs(dir_path, exist_ok=True)
    
    # List of required subdirectories
    subdirs = ['dataset', 'src', 'checkpoints', 'output']
    dirs_to_clear = ['checkpoints', 'output']    
    
    # Create each subdirectory if it doesn’t exist
    for subdir in subdirs:
        subdir_path = os.path.join(dir_path, subdir)
        os.makedirs(subdir_path, exist_ok=True)
    
    # Set permissions recursively for main directory and all subdirectories/files
    for root, dirs, files in os.walk(dir_path):
        os.chmod(root, 0o777)
        for d in dirs:
            os.chmod(os.path.join(root, d), 0o777)
        for f in files:
            os.chmod(os.path.join(root, f), 0o777)
    
    # Clear all contents in the specified directories
    for clear_dir in dirs_to_clear:
        clear_path = os.path.join(dir_path, clear_dir)
        for filename in os.listdir(clear_path):
            file_path = os.path.join(clear_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Remove file or link
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Remove directory and all contents
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')

    return os.path.abspath(dir_path)

def check_file_exists(file_path):
    """
    Check if a file exists at the specified path.

    Parameters:
        file_path (str): The path of the file to check.

    Returns:
        bool: True if the file exists, False otherwise.
    """
    return os.path.isfile(file_path)

def is_archive_file(file_path):
    """
    Check if a file is a valid archive (zip or tar).

    Parameters:
        file_path (str): The path of the file to check.

    Returns:
        bool: True if the file is a valid archive, False otherwise.
    """
    return zipfile.is_zipfile(file_path) or tarfile.is_tarfile(file_path)

In [ ]:
from finn.util.basic import pynq_part_map


def setup_project(prj_name, brd_name, model_type, project_folder=None, model_py_file=None, model_pth_file=None, sample_untrained_model=None, sample_pretrained_model=None, finn_pretrained_model=None, dataset_type=None, custom_dataset=None, torch_vision_dataset=None):
    """
    Set up a project with a specified structure, including creating necessary directories, 
    validating model type, and checking for necessary files.

    Parameters:
        prj_name (str): The name of the project.
        brd_name (str): The name of the target board for the project (e.g., PYNQ board).
        model_type (str): The type of model ('untrained', 'sample_untrained', 'custom_pretrained', 
                          'sample_pretrained', or 'finn_pretrained').
        project_folder (str, optional): The main folder for the project. A new folder is generated if not provided.
        model_py_file (str, optional): The filename of the Python script defining the model architecture, required 
                                       for 'untrained' and 'custom_pretrained' models.
        model_pth_file (str, optional): The filename of the .pth file with pre-trained weights, required for 
                                        'custom_pretrained' models.
        sample_untrained_model (str, optional): The name of the sample untrained model to load, required for 
                                                'sample_untrained' models.
        sample_pretrained_model (str, optional): The name of the sample pretrained model to load, required for 
                                                 'sample_pretrained' models.
        finn_pretrained_model (str, optional): The name of the FINN pretrained model to load, required for 
                                               'finn_pretrained' models.
        dataset_type (str, optional): Type of dataset for 'untrained' models ('torch_vision_dataset' or 
                                      'custom_dataset').
        custom_dataset (str, optional): Path to the custom dataset file for 'untrained' models with 'custom_dataset' 
                                        dataset type.
        torch_vision_dataset (str, optional): Name of the TorchVision dataset class for 'untrained' models with 
                                              'torch_vision_dataset' dataset type.

    Returns:
        dict: A dictionary with project setup information including project name, folder path, model details, 
              and dataset information.
    """
    log_message("Setting up project")
    working_folder = "/home/fastqnn/finn/notebooks/Fast-QNN/outputs/txaviour/"
    prj_info = {}
    
    # Ensure Project Name is provided
    if not prj_name:
        log_message("Project Name is required", level="error")

    # Sanitize project name and set project info
    prj_name_stripped = sanitize_string(prj_name)
    display_name = prj_name
    prj_info["Display_Name"] = display_name
    prj_info["Stripped_Name"] = prj_name_stripped

    if not project_folder:
        # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") # for production mode
        timestamp = "0"
        project_folder = f"{working_folder}{prj_name_stripped}_{timestamp}"
    
    folder_path = ensure_directory_exists(project_folder)
    prj_info["Folder"] = folder_path
    
    available_boards = pynq_part_map.keys()
    if brd_name in available_boards:
        prj_info["Board_name"] = brd_name
    else:
        log_message(f"'{brd_name}' is not a valid board name. Available board names are: {available_boards}", level="error")

    # Validate Model Type
    valid_model_types = ["untrained", "sample_untrained", "custom_pretrained", "sample_pretrained", "finn_pretrained"]
    if model_type not in valid_model_types:
        log_message(f"Model Type must be one of {valid_model_types}", level="error")
    prj_info['Model_Type'] = model_type
    
    # Handle Model Type specific requirements
    if model_type in ["untrained", "custom_pretrained"]:
        if not model_py_file or not check_file_exists(os.path.join(project_folder, 'src', model_py_file)):
            log_message(f"Model Py File '{model_py_file}' does not exist in '{project_folder}'", level="error")
        prj_info["Model_Py_File"] = model_py_file

    if model_type == "custom_pretrained":
        if not model_pth_file or not check_file_exists(os.path.join(project_folder, 'src', model_pth_file)):
            log_message(f"Model Pth File '{model_pth_file}' does not exist in '{project_folder}'", level="error")
        prj_info["Model_Pth_File"] = model_pth_file

    if model_type == "sample_pretrained":
        available_models = ["alexnet_3w3a_cifar10", "alexnet_3w3a_mnist"]
        if not sample_pretrained_model or sample_pretrained_model not in available_models:
            log_message(f"Sample Pretrained Model must be one of: {available_models}", level="error")
        prj_info["Pretrained_Model"] = sample_pretrained_model
        
    if model_type == "sample_untrained":
        available_models = ["alexnet", "resnet"]
        if not sample_untrained_model or sample_untrained_model not in available_models:
            log_message(f"Sample Untrained Model must be one of: {available_models}", level="error")
        prj_info["Sample_Untrained_Model"] = sample_untrained_model

    if model_type == "finn_pretrained":
        available_models = ["cnv_1w1a", "cnv_1w2a", "cnv_2w2a", "lfc_1w1a", "lfc_1w2a", "sfc_1w1a", "sfc_1w2a", "sfc_2w2a", "tfc_1w1a", "tfc_1w2a", "tfc_2w2a", "quant_mobilenet_v1_4b"]
        if not finn_pretrained_model or finn_pretrained_model not in available_models:
            log_message(f"Finn Pretrained Model must be one of: {available_models}", level="error")
        prj_info["Pretrained_Model"] = finn_pretrained_model

    # Handle Dataset requirements for Untrained models
    if model_type == "untrained" or model_type == "sample_untrained":
        if dataset_type not in ["torch_vision_dataset", "custom_dataset"]:
            log_message("Dataset Type must be either 'Torch Vision' or 'Custom Dataset' for Untrained models", level="error")
        prj_info["Dataset_Type"] = dataset_type

        if dataset_type == "custom_dataset":
            custom_dataset_path = os.path.join(project_folder, 'dataset', custom_dataset)
            if not custom_dataset or not check_file_exists(custom_dataset_path) or not is_archive_file(custom_dataset_path):
                log_message(f"Custom Dataset '{custom_dataset}' must exist in '{project_folder}' and be an archive file (zip or tar)", level="error")
            prj_info["Custom_Dataset"] = custom_dataset

        elif dataset_type == "torch_vision_dataset":
            available_datasets = [cls_name.lower() for cls_name in dir(datasets) if not cls_name.startswith('_')]
            if not torch_vision_dataset or torch_vision_dataset.lower() not in available_datasets:
                log_message(f"Torch Vision Dataset must be one of: {available_datasets}", level="error")
            prj_info["Torch_Vision_Dataset"] = torch_vision_dataset
            
    log_message(f"Project setup complete. {prj_name} has been initialized.")
    return prj_info

In [ ]:
import torch
import torch.nn as nn
import brevitas.nn as qnn
from brevitas.core.quant import QuantType

class QuantAlexNet(nn.Module):
    """
    A slightly adapted AlexNet-like architecture for e.g. MNIST-like inputs.
    Uses Brevitas quantization layers (QuantConv2d, QuantLinear, QuantReLU).
    Explicitly sets 'weight_quant_type' and 'quant_type' so that FINN can parse them.
    Layer-by-layer structure (no nn.Sequential).
    """
    def __init__(self, num_bits=1, num_classes=10):
        super(QuantAlexNet, self).__init__()

        # Decide on quantization type based on bit-width
        if num_bits == 1:
            weight_quant_type = QuantType.BINARY
            act_quant_type = QuantType.BINARY
        else:
            weight_quant_type = QuantType.INT
            act_quant_type = QuantType.INT

        # ---------------------------
        #  CONVOLUTIONAL FEATURES
        # ---------------------------

        # 1st Conv block
        self.conv1 = qnn.QuantConv2d(
            in_channels=1, out_channels=64, kernel_size=11, stride=4, padding=2,
            weight_bit_width=num_bits, weight_quant_type=weight_quant_type, bias=False
        )
        self.act1 = qnn.QuantReLU(bit_width=num_bits, quant_type=act_quant_type)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 2nd Conv block
        self.conv2 = qnn.QuantConv2d(
            in_channels=64, out_channels=192, kernel_size=5, padding=2,
            weight_bit_width=num_bits, weight_quant_type=weight_quant_type, bias=False
        )
        self.act2 = qnn.QuantReLU(bit_width=num_bits, quant_type=act_quant_type)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 3rd Conv block
        self.conv3 = qnn.QuantConv2d(
            in_channels=192, out_channels=384, kernel_size=3, padding=1,
            weight_bit_width=num_bits, weight_quant_type=weight_quant_type, bias=False
        )
        self.act3 = qnn.QuantReLU(bit_width=num_bits, quant_type=act_quant_type)

        # 4th Conv block
        self.conv4 = qnn.QuantConv2d(
            in_channels=384, out_channels=256, kernel_size=3, padding=1,
            weight_bit_width=num_bits, weight_quant_type=weight_quant_type, bias=False
        )
        self.act4 = qnn.QuantReLU(bit_width=num_bits, quant_type=act_quant_type)

        # 5th Conv block
        self.conv5 = qnn.QuantConv2d(
            in_channels=256, out_channels=256, kernel_size=3, padding=1,
            weight_bit_width=num_bits, weight_quant_type=weight_quant_type, bias=False
        )
        self.act5 = qnn.QuantReLU(bit_width=num_bits, quant_type=act_quant_type)
        # Skipping final MaxPool because of the small input size (28x28).

        # ---------------------------
        #    CLASSIFIER LAYERS
        # ---------------------------
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc1 = qnn.QuantLinear(
            256, 4096,
            weight_bit_width=num_bits, weight_quant_type=weight_quant_type, bias=False
        )
        self.act_fc1 = qnn.QuantReLU(bit_width=num_bits, quant_type=act_quant_type)

        self.dropout2 = nn.Dropout(p=0.5)
        self.fc2 = qnn.QuantLinear(
            4096, 4096,
            weight_bit_width=num_bits, weight_quant_type=weight_quant_type, bias=False
        )
        self.act_fc2 = qnn.QuantReLU(bit_width=num_bits, quant_type=act_quant_type)

        self.fc3 = qnn.QuantLinear(
            4096, num_classes,
            weight_bit_width=num_bits, weight_quant_type=weight_quant_type, bias=False
        )

    def forward(self, x):
        # 1st Conv block
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)

        # 2nd Conv block
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)

        # 3rd Conv block
        x = self.conv3(x)
        x = self.act3(x)

        # 4th Conv block
        x = self.conv4(x)
        x = self.act4(x)

        # 5th Conv block
        x = self.conv5(x)
        x = self.act5(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Classifier
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.act_fc1(x)

        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.act_fc2(x)

        x = self.fc3(x)
        return x

In [ ]:
from finn.util.test import get_test_model_trained


def load_pretrained_model(model_name, model_type, src_folder, initial_channels = 3, max_size = 4096):
    """
    Loads a pre-trained model from TorchVision and ensures all downloads are in the src folder of the Project.

    Parameters:
        model_type: 
        initial_channels: 
        max_size: 
        model_name (str): The name of the pre-trained model to load (e.g., 'alexnet', 'resnet50').
        src_folder (str): The folder where model downloads will be stored. Default is 'src'.
                
    Returns:
        torch.nn.Module: The loaded pre-trained model.
    """
    log_message(f"Loading {model_type} Model: {model_name}")
    # Ensure the src folder exists
    os.makedirs(src_folder, exist_ok=True)
    # Set TORCH_HOME to the src folder to store the model downloads there
    os.environ['TORCH_HOME'] = src_folder
    pretrained_model = None
    if model_type == "sample_pretrained":
        if model_name == "alexnet_3w3a_cifar10":
            pretrained_model = QuantAlexNet(num_bits=3, num_classes=10).to('cpu')
            pretrained_model.load_state_dict(torch.load("/home/fastqnn/finn/notebooks/Fast-QNN/alexnet_3w3a_cifar10.pth", map_location=torch.device('cpu')))
            pretrained_model.eval()
        if model_name == "alexnet_3w3a_mnist":
            pretrained_model = QuantAlexNet(num_bits=3, num_classes=10).to('cpu')
            pretrained_model.load_state_dict(torch.load("/home/fastqnn/finn/notebooks/Fast-QNN/alexnet_3w3a_mnist.pth", map_location=torch.device('cpu')))
            pretrained_model.eval()
    elif model_type == "finn_pretrained":
        if model_name == "cnv_1w1a":
            pretrained_model = get_test_model_trained("CNV", 1, 1)
        elif model_name == "cnv_1w2a":
            pretrained_model = get_test_model_trained("CNV", 1, 2)
        elif model_name == "cnv_2w2a":
            pretrained_model = get_test_model_trained("CNV", 2, 2)
        elif model_name == "lfc_1w1a":
            pretrained_model = get_test_model_trained("LFC", 1, 1)
        elif model_name == "lfc_1w2a":
            pretrained_model = get_test_model_trained("LFC", 1, 2)
        elif model_name == "sfc_1w1a":
            pretrained_model = get_test_model_trained("SFC", 1, 1)
        elif model_name == "sfc_1w2a":
            pretrained_model = get_test_model_trained("SFC", 1, 2)
        elif model_name == "sfc_2w2a":
            pretrained_model = get_test_model_trained("SFC", 2, 2)
        elif model_name == "tfc_1w1a":
            pretrained_model = get_test_model_trained("TFC", 1, 1)
        elif model_name == "tfc_1w2a":
            pretrained_model = get_test_model_trained("TFC", 1, 2)
        elif model_name == "tfc_2w2a":
            pretrained_model = get_test_model_trained("TFC", 2, 2)
        elif model_name == "quant_mobilenet_v1_4b":
            pretrained_model = get_test_model_trained("mobilenet", 4, 4)
    # List of common input shapes to test first (both square and non-square)
    common_shapes = [
        (1, initial_channels, 32, 32),  # Typical for many models
        (1, initial_channels, 28, 28),  # Typical for many models
        (1, initial_channels, 224, 224),  # Typical for many models
        (1, initial_channels, 299, 299),  # For models like Inception
        (1, initial_channels, 128, 128),  # Smaller size
        (1, initial_channels, 256, 256),  # Larger square size
        (1, initial_channels, 320, 240),  # Common non-square size
        (1, initial_channels, 240, 320),  # Non-square (swapped dimensions)
        (1, initial_channels, 256, 128),  # Non-square
        (1, initial_channels, 128, 256),  # Non-square (swapped)
    ]

    # Try common shapes first
    for shape in common_shapes:
        try:
            dummy_input = torch.rand(*shape)
            pretrained_model(dummy_input)
            log_message(f"Compatible common input shape found: {shape}")
            return pretrained_model, shape
        except RuntimeError:
            continue

    # If no common shape worked, test all possible square and non-square shapes up to max_size
    for width in range(1, max_size + 1, 1):  # Step by 16 for efficiency
        for height in range(1, max_size + 1, 1):
            try:
                dummy_input = torch.rand(1, initial_channels, width, height)
                pretrained_model(dummy_input)
                pretrained_model_input_shape = (1, initial_channels, width, height)
                log_message(f"Compatible input shape found: {pretrained_model_input_shape}")
                return pretrained_model, pretrained_model_input_shape
            except RuntimeError:
                continue

    log_message("Could not determine a compatible input shape within the specified range.", level="error")

In [ ]:
prj_name_input = "Alexnet 3w3a MNIST"
board_name_input = "Pynq-Z2"
prj_folder_input = sanitize_string(prj_name_input)
model_type_input = "sample_pretrained"
pretrained_model_name_input = "alexnet_3w3a_mnist"
Project_Info = setup_project(prj_name=prj_name_input, brd_name=board_name_input, model_type=model_type_input, sample_pretrained_model=pretrained_model_name_input)

input_model= None
input_model_shape = None

if Project_Info['Model_Type'] == "untrained":
    log_message("Training for untrained models are not supported at the moment!", level="error")
elif Project_Info['Model_Type'] == "custom_pretrained":
    log_message("Custom Pretrained models are not supported at the moment!", level="error")
elif Project_Info['Model_Type'] == "sample_pretrained" or Project_Info['Model_Type'] == "finn_pretrained":
    pretrained_folder = os.path.join(Project_Info['Folder'],"src")
    input_model, input_model_shape = load_pretrained_model(Project_Info['Pretrained_Model'], Project_Info['Model_Type'], pretrained_folder, initial_channels=1)
else:
    log_message("Unsupported Model Type", level="error")

In [ ]:
def set_onnx_checkpoint(project_info, suffix):
    """
    Generates the export path for ONNX files based on a specified suffix.

    Parameters:
        project_info (dict): Dictionary containing project information (e.g., 'Folder' and 'Stripped_Name').
        suffix (str): The suffix to append to the exported ONNX filename (e.g., "model1" for "model1_export.onnx").

    Returns:
        str: The full path to the export file.
    """
    log_message(f"Saving Checkpoint: {suffix}")
    suffix = sanitize_string(suffix)
    filename = f"{project_info['Stripped_Name']}_{suffix}.onnx"
    return os.path.join(project_info['Folder'], "checkpoints", filename)


In [ ]:
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup

export_onnx_path = set_onnx_checkpoint(Project_Info,"Brevitas Export")
export_qonnx(input_model, torch.randn(input_model_shape), export_onnx_path, opset_version=9)
qonnx_cleanup(export_onnx_path, out_file=export_onnx_path)

In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

model = ModelWrapper(export_onnx_path)
model = model.transform(ConvertQONNXtoFINN())
model.save(set_onnx_checkpoint(Project_Info,"QONNX to FINN"))

In [ ]:
from qonnx.transformation.double_to_single_float import DoubleToSingleFloat
from qonnx.transformation.remove import RemoveIdentityOps
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import (GiveReadableTensorNames,
                                          GiveUniqueNodeNames,
                                          RemoveStaticGraphInputs,
                                          RemoveUnusedTensors, GiveUniqueParameterTensors, SortGraph)
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_data_layouts import InferDataLayouts

def tidy_up_transforms(input_tidy_model, save_name):
    """
    Applies a series of transformations to a model and saves the resulting model.

    Parameters:
        input_tidy_model (ModelWrapper): The model to transform.
        save_name (str): The path to save the transformed model.
    
    Returns:
        ModelWrapper: The transformed model.
    """

    # Apply transformations
    input_tidy_model = input_tidy_model.transform(GiveUniqueParameterTensors())
    input_tidy_model = input_tidy_model.transform(InferShapes())
    input_tidy_model = input_tidy_model.transform(FoldConstants())
    input_tidy_model = input_tidy_model.transform(GiveUniqueNodeNames())
    input_tidy_model = input_tidy_model.transform(GiveReadableTensorNames())
    input_tidy_model = input_tidy_model.transform(InferDataTypes())
    input_tidy_model = input_tidy_model.transform(RemoveStaticGraphInputs())

    # Save the transformed model
    input_tidy_model.save(save_name)

    return input_tidy_model

In [ ]:
model = tidy_up_transforms(model, set_onnx_checkpoint(Project_Info,"Tidy ONNX Post Finn"))

In [ ]:
from qonnx.core.datatype import DataType
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from finn.util.pytorch import ToTensor

log_message("Skipping Pre-Processing. Will be expecting the user to handle it in application!", level="warning")

In [ ]:
"""
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = set_onnx_checkpoint(Project_Info,"Pre Proc ONNX Finn")
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name, opset_version=9)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])
"""

In [ ]:
import torch.nn as nn

class MultiplyByOne(nn.Module):
    def forward(self, x):
        out = x * 2
        out = out + 0  # Adding a no-op to prevent graph simplification
        out = out / 2  # Adding a no-op to prevent graph simplification
        return out

global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
mul_model = MultiplyByOne()

chkpt_mul_name = set_onnx_checkpoint(Project_Info, "Mul_One_ONNX")
export_qonnx(mul_model, torch.randn(ishape), chkpt_mul_name, opset_version=9)
qonnx_cleanup(chkpt_mul_name, out_file=chkpt_mul_name)

mul_model_wrapper = ModelWrapper(chkpt_mul_name)
mul_model_wrapper = mul_model_wrapper.transform(ConvertQONNXtoFINN())
model = model.transform(MergeONNXModels(mul_model_wrapper))

global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

In [ ]:
from qonnx.transformation.insert_topk import InsertTopK

model = model.transform(InsertTopK(k=1))
model.save(set_onnx_checkpoint(Project_Info,"Post Processing"))
model = tidy_up_transforms(model, set_onnx_checkpoint(Project_Info,"Tidy Post PrePost Proc"))

In [ ]:
import numpy as np
from finn.transformation.qonnx.qonnx_activation_handlers import QuantActBaseHandler, np_default_dtype
from qonnx.custom_op.registry import getCustomOp

class ReluToMultiThresholdHandler(QuantActBaseHandler):
    """Class for converting a regular non-quantized ReLU operation to a MultiThreshold node in Xilinx FINN."""
    def _check_compatibility(self):
        # Check if the ReLU operation is supported for transformation
        pass

    def _calculate_act_bias(self):
        # No bias is applied for standard ReLU
        return np.array([0.0], dtype=np_default_dtype)

    def _calculate_thresholds(self):
        # Thresholds for ReLU are typically defined as a single zero threshold
        output_shape = self._model.get_tensor_shape(self._q_node.output[0])
        num_output_channels = output_shape[1]  # Assuming NCHW or NHWC format
        thresholds = np.zeros((num_output_channels, 1), dtype=np_default_dtype)
        return thresholds

    def _calculate_act_scale(self):
        # Standard ReLU does not apply scaling
        return np.array([1.0], dtype=np_default_dtype)

    def _remove_activation_node(self, multi_threshold_node):
        # Find the activation node
        act_node = self._model.find_direct_predecessors(self._q_node)
        if act_node is None:
            raise RuntimeError(
                "For handling of Relu activations a predecesor to dgdgdg"
            )
        act_node = act_node[0]
        if act_node.op_type != "Relu":
            raise RuntimeError(
                "The predecesor of the Quant node must be Relu for handling "
                "of activations."
            )

        # Remove the activation node
        self._model.graph.node.remove(act_node)
        return multi_threshold_node

    def calculate_node_parameters(self):
        # Calculate all parameters for replacing ReLU with MultiThreshold
        return {
            "out_dtype": "FLOAT32",  # Regular ReLU typically works with float32
            "thresholds": self._calculate_thresholds(),
            "adder_bias": self._calculate_act_bias(),
            "mul_scale": self._calculate_act_scale(),
        }

    def replace_relu_node(self):
        """Replace a regular ReLU operation with a MultiThreshold node."""
        # Check compatibility
        self._check_compatibility()

        # Shorten instance variables
        model = self._model
        graph = model.graph
        n = self._q_node
        running_node_index = self._q_index

        # Calculate parameters for the MultiThreshold node
        parameter_dict = self.calculate_node_parameters()
        thresholds = parameter_dict["thresholds"]
        adder_bias = parameter_dict["adder_bias"]
        mul_scale = parameter_dict["mul_scale"]
        out_dtype = parameter_dict["out_dtype"]

        # Create threshold tensor
        thresh_tensor = oh.make_tensor_value_info(
            model.make_new_valueinfo_name(),
            TensorProto.FLOAT,
            thresholds.shape,
        )
        graph.value_info.append(thresh_tensor)
        model.set_initializer(thresh_tensor.name, thresholds)
        act_node = self._model.find_direct_predecessors(self._q_node)
        act_node = act_node[0]
        # Insert MultiThreshold node
        mt_node = oh.make_node(
            "MultiThreshold",
            [act_node.input[0], thresh_tensor.name],
            [n.output[0]],
            out_dtype=out_dtype,
            domain="qonnx.custom_op.general",
        )
        graph.node.insert(running_node_index, mt_node)
        running_node_index += 1

        # Get MultiThreshold instance
        mt_inst = getCustomOp(graph.node[running_node_index - 1])

        # Set bias and scale attributes
        mt_inst.set_nodeattr("out_scale", mul_scale[0].item())
        mt_inst.set_nodeattr("out_bias", adder_bias[0].item())
        mt_inst.set_nodeattr("out_dtype", out_dtype)

        # Remove the original ReLU node
        self._remove_activation_node(mt_node)
        graph.node.remove(n)

        # Return the updated model
        return self._model


In [ ]:
from finn.transformation.qonnx.qonnx_activation_handlers import QuantReluHandler
from onnx import TensorProto
from qonnx.transformation.base import Transformation
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from onnx import helper as oh

class CustomReLU(nn.Module):
    def forward(self, x):
        # Implementing an approximation of abs(x) using only arithmetic operations
        epsilon = 1e-6  # Small constant to avoid division by zero
        abs_x = (x * x + epsilon) / (x + epsilon)
        return (x + abs_x) / 2

class CustomReluBreakdown(Transformation):
    """
    Custom transformation to iterate through the nodes, identify ReLU nodes,
    and convert the next node to a quant node while applying the QuantReluHandler.
    """

    def __init__(self, bit_width=1, quant_type=DataType["UINT8"], narrow=0, signed=0):
        super().__init__()
        self.bit_width = bit_width
        self.quant_type = quant_type
        self.narrow = narrow
        self.signed = signed

    def apply(self, transform_model):
        graph = transform_model.graph
        graph_modified = False
        convert_next_node = False

        for idx, node in enumerate(graph.node):
            if convert_next_node:
                # Call QuantReluHandler on the current Quant node
                handler = ReluToMultiThresholdHandler(transform_model, node, idx)
                handler.replace_relu_node()

                convert_next_node = False
                graph_modified = True

            if node.op_type == "Relu":
                # Raise a flag to convert the next node to a quant node
                convert_next_node = True

       # transform_model = transform_model.transform(InferShapes())
        return (transform_model, graph_modified)


In [ ]:
from onnx import helper as oh
from onnx import TensorProto
from qonnx.transformation.base import Transformation
from qonnx.custom_op.registry import getCustomOp
from qonnx.transformation.infer_shapes import InferShapes
import numpy as np


class ReluToMultiThresholdTransform(Transformation):
    """
    Custom transformation to replace ReLU nodes with MultiThreshold nodes
    in Xilinx FINN. Works directly on ReLU nodes and reroutes their inputs/outputs.
    """

    def __init__(self):
        super().__init__()

    def apply(self, transform_model):
        graph = transform_model.graph
        graph_modified = False

        for idx, node in enumerate(graph.node):
            if node.op_type == "Relu":
                # Replace the ReLU node with a MultiThreshold node
                self.replace_relu_with_multithreshold(transform_model, node, idx)
                graph_modified = True

        return (transform_model, graph_modified)

    def replace_relu_with_multithreshold(self, transform_model, relu_node, relu_index):
        """
        Replaces a ReLU node with a MultiThreshold node.
        """
        graph = transform_model.graph

        # Get the input to the ReLU node (its predecessor)
        if len(relu_node.input) != 1:
            raise RuntimeError(f"ReLU node {relu_node.name} has unexpected inputs.")
        relu_input = relu_node.input[0]

        # Get the output of the ReLU node (its successor)
        if len(relu_node.output) != 1:
            raise RuntimeError(f"ReLU node {relu_node.name} has unexpected outputs.")
        relu_output = relu_node.output[0]

        # Calculate parameters for MultiThreshold
        thresholds = self._calculate_thresholds(transform_model, relu_node)
        adder_bias = self._calculate_act_bias()
        mul_scale = self._calculate_act_scale()
        out_dtype = "FLOAT32"

        # Create threshold tensor
        thresh_tensor = oh.make_tensor_value_info(
            transform_model.make_new_valueinfo_name(),
            TensorProto.FLOAT,
            thresholds.shape,
        )
        graph.value_info.append(thresh_tensor)
        transform_model.set_initializer(thresh_tensor.name, thresholds)

        # Create MultiThreshold node
        mt_node = oh.make_node(
            "MultiThreshold",
            inputs=[relu_input, thresh_tensor.name],
            outputs=[relu_output],
            domain="qonnx.custom_op.general",
        )
        mt_node.attribute.extend([
            oh.make_attribute("out_scale", float(mul_scale[0])),
            oh.make_attribute("out_bias", float(adder_bias[0])),
            oh.make_attribute("out_dtype", out_dtype),
        ])

        # Insert MultiThreshold node in place of ReLU node
        graph.node.insert(relu_index, mt_node)

        # Remove the original ReLU node
        graph.node.remove(relu_node)

    def _calculate_thresholds(self, transform_model, relu_node):
        """
        Calculates the thresholds for the MultiThreshold node based on the ReLU operation.
        """
        # ReLU threshold is always zero
        output_shape = transform_model.get_tensor_shape(relu_node.output[0])
        num_output_channels = output_shape[1]  # Assuming NCHW or NHWC format
        thresholds = np.zeros((num_output_channels, 1), dtype=np.float32)
        return thresholds

    def _calculate_act_bias(self):
        """
        Calculates the bias for the MultiThreshold node.
        """
        # ReLU does not apply a bias
        return np.array([0.0], dtype=np.float32)

    def _calculate_act_scale(self):
        """
        Calculates the scale for the MultiThreshold node.
        """
        # ReLU does not apply scaling
        return np.array([1.0], dtype=np.float32)


In [ ]:
import traceback
from qonnx.transformation.batchnorm_to_affine import BatchNormToAffine
from qonnx.transformation.general import ConvertSubToAdd, ConvertDivToMul
from finn.transformation.streamline import Streamline, RoundAndClipThresholds, CollapseRepeatedMul, ConvertSignToThres, \
    CollapseRepeatedAdd
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
from finn.transformation.streamline.absorb import (AbsorbTransposeIntoMultiThreshold,
                                                   AbsorbScalarMulAddIntoTopK,
                                                   AbsorbSignBiasIntoMultiThreshold,
                                                   AbsorbAddIntoMultiThreshold,
                                                   AbsorbMulIntoMultiThreshold, FactorOutMulSignMagnitude,
                                                   Absorb1BitMulIntoMatMul, Absorb1BitMulIntoConv)
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants, MoveAddPastMul, \
    MoveScalarAddPastMatMul, MoveAddPastConv, MoveScalarMulPastMatMul, MoveScalarMulPastConv, \
    MoveMaxPoolPastMultiThreshold, MoveLinearPastEltwiseAdd, MoveLinearPastFork
from finn.transformation.streamline.absorb import AbsorbConsecutiveTransposes

def streamline_transforms(input_streamline_model, save_name):
    """
    Applies a series of streamlining transformations to a model and saves the resulting model.
    If a transformation fails, it logs the error and moves on to the next transformation.

    Parameters:
        input_streamline_model (ModelWrapper): The model to transform.
        save_name (str): The path to save the transformed model.
    
    Returns:
        ModelWrapper: The transformed model.
    """
    transformations = [
        AbsorbSignBiasIntoMultiThreshold(),
        ConvertSubToAdd(),
        ConvertDivToMul(),
        CollapseRepeatedMul(),
        BatchNormToAffine(),
        ConvertSignToThres(),
        MoveAddPastMul(),
        MoveScalarAddPastMatMul(),
        MoveAddPastConv(),
        MoveScalarMulPastMatMul(),
        MoveScalarMulPastConv(),
        MoveAddPastMul(),
        MoveScalarLinearPastInvariants(),
        CollapseRepeatedAdd(),
        AbsorbAddIntoMultiThreshold(),
        FactorOutMulSignMagnitude(),
        MoveMaxPoolPastMultiThreshold(),
        AbsorbMulIntoMultiThreshold(),
        Absorb1BitMulIntoMatMul(),
        Absorb1BitMulIntoConv(),
        AbsorbMulIntoMultiThreshold(),
        Streamline(),
        LowerConvsToMatMul(),
        MakeMaxPoolNHWC(),
        AbsorbTransposeIntoMultiThreshold(),
        ConvertBipolarMatMulToXnorPopcount(),
        Streamline(),
        AbsorbScalarMulAddIntoTopK(),
        RoundAndClipThresholds(),
        MoveLinearPastEltwiseAdd(),
        MoveLinearPastFork()
    ]
    
    input_streamline_model = input_streamline_model.transform(ReluToMultiThresholdTransform())
    input_streamline_model = tidy_up_transforms(input_streamline_model, save_name)
    for iter_id in range(3):
        for transform in transformations:
            try:
                input_streamline_model = input_streamline_model.transform(transform)
            except Exception as e:
                print(e)
                
        input_streamline_model = tidy_up_transforms(input_streamline_model, save_name)
        
        input_streamline_model = input_streamline_model.transform(InferDataLayouts())
        input_streamline_model = input_streamline_model.transform(RemoveUnusedTensors())
        input_streamline_model = input_streamline_model.transform(DoubleToSingleFloat())
        input_streamline_model = input_streamline_model.transform(SortGraph())
        input_streamline_model = input_streamline_model.transform(RemoveIdentityOps())
    
    return input_streamline_model

In [ ]:
model = streamline_transforms(model, set_onnx_checkpoint(Project_Info,"Streamlined ONNX"))

In [ ]:
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.transformation.fpgadataflow.convert_to_hw_layers import (
    InferBinaryMatrixVectorActivation,
    InferQuantizedMatrixVectorActivation,
    InferLabelSelectLayer,
    InferThresholdingLayer,
    InferStreamingMaxPool,
    InferConvInpGen,
    InferAddStreamsLayer,
    InferChannelwiseLinearLayer,
    InferConcatLayer,
    InferDuplicateStreamsLayer,
    InferGlobalAccPoolLayer,
    InferLookupLayer,
    InferPool,
    InferStreamingEltwise,
    InferUpsample,
    InferVectorVectorActivation
)

def to_hw_transforms(input_hw_model, save_name):
    """
    Applies a comprehensive series of hardware-oriented transformations to a model and saves the resulting model.
    If a transformation fails, it logs the error and moves on to the next transformation.

    Parameters:
        input_hw_model (ModelWrapper): The model to transform.
        save_name (str): The path to save the transformed model.
    
    Returns:
        ModelWrapper: The transformed model.
    """
    transformations = [
        InferBinaryMatrixVectorActivation(),
        InferQuantizedMatrixVectorActivation(),
        InferLabelSelectLayer(),
        InferThresholdingLayer(),
        InferConvInpGen(),
        InferStreamingMaxPool(),
        InferAddStreamsLayer(),
        InferChannelwiseLinearLayer(),
        InferConcatLayer(),
        InferDuplicateStreamsLayer(),
        InferGlobalAccPoolLayer(),
        InferLookupLayer(),
        InferPool(),
        InferStreamingEltwise(),
        InferUpsample(),
        InferVectorVectorActivation(),
        RemoveCNVtoFCFlatten(),
        AbsorbConsecutiveTransposes()
    ]

    for iter_id in range(3):
        for transform in transformations:
            try:
                input_hw_model = input_hw_model.transform(transform)
            except Exception as e:
                print(e)
                pass

        # Apply final tidy-up transformations
        input_hw_model = tidy_up_transforms(input_hw_model, save_name)
        
        input_hw_model = input_hw_model.transform(InferDataLayouts())
        input_hw_model = input_hw_model.transform(RemoveUnusedTensors())
        input_hw_model = input_hw_model.transform(DoubleToSingleFloat())
        input_hw_model = input_hw_model.transform(SortGraph())
        input_hw_model = input_hw_model.transform(RemoveIdentityOps())

    # Save the final transformed model
    input_hw_model.save(save_name)

    return input_hw_model

In [ ]:
model = to_hw_transforms(model, set_onnx_checkpoint(Project_Info,"To HW Layers"))

In [ ]:
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)

def dataflow_partitioning(input_data_model, save_name):
    """
    Applies dataflow partitioning transformation to the model and saves the resulting parent and dataflow models.

    Parameters:
        input_data_model (ModelWrapper): The model to transform.
        save_name (str): The directory to save the transformed models.

    Returns:
        ModelWrapper: The transformed parent model.
    """
    # Apply dataflow partitioning
    parent_model = input_data_model.transform(CreateDataflowPartition())
    parent_model.save(save_name)
    
    # Retrieve the dataflow partition model filename
    sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
    sdp_node = getCustomOp(sdp_node)
    dataflow_model_filename = sdp_node.get_nodeattr("model")
    # Load and return the dataflow model
    dataflow_model = ModelWrapper(dataflow_model_filename)
    
    return dataflow_model

In [ ]:
model = dataflow_partitioning(model, set_onnx_checkpoint(Project_Info,"Dataflow Partition Parent Model"))
model.save(set_onnx_checkpoint(Project_Info,"Dataflow Partition Streaming Model"))

In [ ]:
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers

def specialize_layers_transform(input_specialize_model, board_name, save_name):
    """
    Applies layer specialization transformation to a dataflow model for the specified FPGA part and saves the resulting model.

    Parameters:
        input_specialize_model (ModelWrapper): The dataflow model to transform.
        board_name (str): The FPGA board for which to specialize the layers.
        save_name (str): The path to save the specialized model.
    
    Returns:
        ModelWrapper: The transformed and specialized dataflow model.
    """
    fpga_part = pynq_part_map[board_name]
    # Apply specialization for FPGA layers
    input_specialize_model = input_specialize_model.transform(SpecializeLayers(fpga_part))

    # Save the specialized model
    input_specialize_model.save(save_name)

    return input_specialize_model

In [ ]:
model = specialize_layers_transform(model, Project_Info['Board_name'], set_onnx_checkpoint(Project_Info,f"Specialize Model Layers to {Project_Info['Board_name']}"))

In [ ]:
# model = ModelWrapper("/home/fastqnn/finn/notebooks/Fast-QNN/outputs/txaviour/alexnet_3w3a_mnist_0/checkpoints/alexnet_3w3a_mnist_specialize_model_layers_to_pynqzu.onnx")

In [ ]:
import math
import os
from finn.transformation.fpgadataflow.set_folding import SetFolding
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from finn.transformation.fpgadataflow.insert_iodma import InsertIODMA
from qonnx.custom_op.registry import getCustomOp
from functools import partial
import matplotlib.pyplot as plt

def get_int_attr(node_inst, attr_name = None):
    """
    Safely retrieves an integer attribute from a node.

    Parameters:
        node_inst: The node instance.
        attr_name (str): The name of the attribute to retrieve.

    Returns:
        int: The integer value of the attribute.
    """
    if attr_name is not None:
        attr = node_inst.get_nodeattr(attr_name)
    else:
        attr = node_inst
    if isinstance(attr, (tuple, list)):
        if len(attr) == 0:
            raise ValueError(f"Attribute '{attr_name}' for node '{node_inst.name}' is an empty tuple/list.")
        return get_int_attr(attr[0])
    elif isinstance(attr, int):
        return attr
    else:
        raise TypeError(f"Attribute '{attr_name}' for node '{node_inst.name}' is of unsupported type {type(attr)}.")


from qonnx.custom_op.registry import getCustomOp

def generate_valid_folding_factors(folding_model, max_pe=32, max_simd=32, max_bitwidth=8191, min_infifodepth=8, max_infifodepth=128):
    """
    Generates a folding configuration dictionary ensuring all constraints are met,
    including specific SIMD requirements from HLS synthesis.

    Parameters:
        max_infifodepth:
        min_infifodepth:
        folding_model (ModelWrapper): The FINN model.
        max_pe (int): Maximum allowed PE value.
        max_simd (int): Maximum allowed SIMD value.
        max_bitwidth (int): Maximum allowed stream width in bits.

    Returns:
        dict: A dictionary with node names as keys and their folding parameters as values.
    """
    folding_config = {}

    # Process MVAU_hls layers
    fc_layers = folding_model.get_nodes_by_op_type("MVAU_hls")
    for node in fc_layers:
        node_inst = getCustomOp(node)
        node_name = node.name

        # Safely retrieve attributes
        try:
            MW = get_int_attr(node_inst, "MW")  # Number of input features
            MH = get_int_attr(node_inst, "MH")  # Number of output features
        except (ValueError, TypeError) as e:
            log_message(f"Error retrieving attributes for node '{node_name}': {e}", "error")
            continue  # Skip this node or handle as needed

        input_bitwidth = node_inst.get_input_datatype().bitwidth()
        output_bitwidth = node_inst.get_output_datatype().bitwidth()

        # Calculate minimum required SIMD based on HLS constraint
        min_simd = math.ceil(MW / 1024)

        # Determine valid SIMD: largest divisor of MW <= max_simd and ensures (SIMD * input_bitwidth) <= max_bitwidth
        simd_candidates = [
            x for x in range(min_simd, min(max_simd, MW) + 1)
            if MW % x == 0 and (x * input_bitwidth) <= max_bitwidth
        ]
        simd = max(simd_candidates) if simd_candidates else min_simd  # Ensure at least min_simd

        # Determine valid PE: largest divisor of MH <= max_pe and ensures (PE * output_bitwidth) <= max_bitwidth
        pe_candidates = [
            x for x in range(1, min(max_pe, MH) + 1)
            if MH % x == 0 and (x * output_bitwidth) <= max_bitwidth
        ]
        pe = max(pe_candidates) if pe_candidates else 1

        if pe * simd > 16:
            infifodepth = min(max_infifodepth, 32)  # Example: Set to 32 if high parallelism
        else:
            infifodepth = max(min_infifodepth, 8)

        max_possible_infifodepth = max_bitwidth // (simd * input_bitwidth)
        infifodepth = min(infifodepth, max_possible_infifodepth)
        infifodepth = max(infifodepth, min_infifodepth)

        log_message(f"Node '{node_name}': Selected infifodepth={infifodepth}")

        folding_config[node_name] = {
            "PE": pe,
            "SIMD": simd,
            "infifodepth": infifodepth
        }
        log_message(f"Configured node '{node_name}' with PE={pe}, SIMD={simd}, infifodepth={infifodepth}")

    # Process VVAU layers if any (similar logic can be applied)
    vvau_layers = folding_model.get_nodes_by_op_type("VVAU")
    for node in vvau_layers:
        node_inst = getCustomOp(node)
        node_name = node.name

        # Safely retrieve attributes
        try:
            MW = get_int_attr(node_inst, "MW")  # Number of input features
            MH = get_int_attr(node_inst, "MH")  # Number of output features
        except (ValueError, TypeError) as e:
            log_message(f"Error retrieving attributes for node '{node_name}': {e}", "error")
            continue  # Skip this node or handle as needed

        input_bitwidth = node_inst.get_input_datatype().bitwidth()
        output_bitwidth = node_inst.get_output_datatype().bitwidth()

        # Calculate minimum required SIMD based on HLS constraint
        min_simd = math.ceil(MW / 1024)

        # Determine valid SIMD and PE, ensuring SWG SIMD == VVAU PE if window-parallelism is enabled
        simd_candidates = [
            x for x in range(min_simd, min(max_simd, MW) + 1)
            if MW % x == 0 and (x * input_bitwidth) <= max_bitwidth
        ]
        simd = max(simd_candidates) if simd_candidates else min_simd  # Ensure at least min_simd

        pe_candidates = [
            x for x in range(1, min(max_pe, MH) + 1)
            if MH % x == 0 and (x * output_bitwidth) <= max_bitwidth
        ]
        pe = max(pe_candidates) if pe_candidates else 1

        # Ensure SWG SIMD == VVAU PE (as per VVAU constraints in window-parallelism)
        pe = simd

        if pe * simd > 16:
            infifodepth = min(max_infifodepth, 32)  # Example: Set to 32 if high parallelism
        else:
            infifodepth = max(min_infifodepth, 8)

        max_possible_infifodepth = max_bitwidth // (simd * input_bitwidth)
        infifodepth = min(infifodepth, max_possible_infifodepth)
        infifodepth = max(infifodepth, min_infifodepth)

        log_message(f"Node '{node_name}': Selected infifodepth={infifodepth}")

        folding_config[node_name] = {
            "PE": pe,
            "SIMD": simd,
            "infifodepth": infifodepth
        }
        log_message(f"Configured node '{node_name}' with PE={pe}, SIMD={simd}, infifodepth={infifodepth}")

    return folding_config

def apply_folding_config(folding_model, folding_config):
    """
    Applies the folding configuration to the model.

    Parameters:
        folding_model (ModelWrapper): The FINN model.
        folding_config (dict): Folding configuration dictionary.

    Returns:
        ModelWrapper: The model with updated folding parameters.
    """
    # Apply folding to MVAU_hls layers
    fc_layers = folding_model.get_nodes_by_op_type("MVAU_hls")
    for node in fc_layers:
        node_name = node.name
        if node_name in folding_config:
            config = folding_config[node_name]
            node_inst = getCustomOp(node)
            node_inst.set_nodeattr("PE", config["PE"])
            node_inst.set_nodeattr("SIMD", config["SIMD"])
            log_message(f"Updated node '{node_name}' with PE={config['PE']} and SIMD={config['SIMD']}")

    # Apply folding to VVAU layers
    vvau_layers = folding_model.get_nodes_by_op_type("VVAU")
    for node in vvau_layers:
        node_name = node.name
        if node_name in folding_config:
            config = folding_config[node_name]
            node_inst = getCustomOp(node)
            node_inst.set_nodeattr("PE", config["PE"])
            node_inst.set_nodeattr("SIMD", config["SIMD"])
            log_message(f"Updated node '{node_name}' with PE={config['PE']} and SIMD={config['SIMD']}")

    return folding_model

def validate_stream_widths(folding_model, max_bitwidth=8191):
    """
    Validates that all stream widths are within the allowed maximum.

    Parameters:
        folding_model (ModelWrapper): The FINN model.
        max_bitwidth (int): Maximum allowed stream width in bits.

    Raises:
        AssertionError: If any stream width exceeds max_bitwidth.
    """
    # Check MVAU_hls layers
    fc_layers = folding_model.get_nodes_by_op_type("MVAU_hls")
    for node in fc_layers:
        node_inst = getCustomOp(node)
        simd = node_inst.get_nodeattr("SIMD")
        pe = node_inst.get_nodeattr("PE")
        input_bitwidth = node_inst.get_input_datatype().bitwidth()
        output_bitwidth = node_inst.get_output_datatype().bitwidth()

        instream_width = simd * input_bitwidth
        outstream_width = pe * output_bitwidth

        assert instream_width <= max_bitwidth, f"{node.name} has input stream width {instream_width} bits > {max_bitwidth}"
        assert outstream_width <= max_bitwidth, f"{node.name} has output stream width {outstream_width} bits > {max_bitwidth}"

    # Check VVAU layers if any
    vvau_layers = folding_model.get_nodes_by_op_type("VVAU")
    for node in vvau_layers:
        node_inst = getCustomOp(node)
        simd = node_inst.get_nodeattr("SIMD")
        pe = node_inst.get_nodeattr("PE")
        input_bitwidth = node_inst.get_input_datatype().bitwidth()
        output_bitwidth = node_inst.get_output_datatype().bitwidth()

        instream_width = simd * input_bitwidth
        outstream_width = pe * output_bitwidth

        assert instream_width <= max_bitwidth, f"{node.name} has input stream width {instream_width} bits > {max_bitwidth}"
        assert outstream_width <= max_bitwidth, f"{node.name} has output stream width {outstream_width} bits > {max_bitwidth}"

    print("All stream widths are within the allowed maximum.")

def insert_dwcs_if_needed(model_dwc):
    """
    Inserts DataWidthConverters (DWCs) where stream widths between consecutive layers do not match.

    Parameters:
        model_dwc (ModelWrapper): The FINN model.

    Returns:
        ModelWrapper: The model with DWCs inserted where necessary.
    """

    # This is a simplified approach. Depending on your model, you might need a more sophisticated check.
    # InsertStreamConverter handles the insertion of DWCs automatically when stream widths mismatch.
    model_dwc = model_dwc.transform(InsertDWC())

    return model_dwc



In [ ]:
def folding_transform(input_folding_model: ModelWrapper, save_name: str, **kwargs):
    """
    Applies a user-defined folding configuration (from generate_folding_config) to MVAU_hls and
    ConvolutionInputGenerator_rtl layers, then saves the result.

    Parameters:
        input_folding_model (ModelWrapper): The specialized model to transform.
        save_name (str): Path to save the transformed model (e.g., checkpoint).
        **kwargs: Pass additional arguments to generate_folding_config (e.g. max_pe, max_simd, etc.)

    Returns:
        ModelWrapper: The transformed and folded dataflow model.
    """
    max_pe=16
    max_simd=16
    max_bitwidth=8191
    min_infifodepth=8
    max_infifodepth=128
    # Generate folding config dictionary keyed by node name
    input_folding_model = input_folding_model.transform(GiveUniqueNodeNames())

    # Generate folding configuration
    folding_config = generate_valid_folding_factors(input_folding_model, max_pe=max_pe, max_simd=max_simd, max_bitwidth=max_bitwidth, min_infifodepth=min_infifodepth, max_infifodepth=max_infifodepth)

    # Apply folding configuration
    input_folding_model = apply_folding_config(input_folding_model, folding_config)

    # Validate stream widths
    try:
        validate_stream_widths(input_folding_model, max_bitwidth=max_bitwidth)
    except AssertionError as e:
        print(f"Stream width validation failed: {e}")
        # Optionally, you can implement a fallback strategy here
        # For simplicity, we'll reduce SIMD and PE by half and retry
        print("Reducing max_simd and max_pe by half and retrying...")
        new_max_simd = max(1, max_simd // 2)
        new_max_pe = max(1, max_pe // 2)
        folding_config = generate_valid_folding_factors(input_folding_model, max_pe=new_max_pe, max_simd=new_max_simd, max_bitwidth=max_bitwidth)
        input_folding_model = apply_folding_config(input_folding_model, folding_config)
        validate_stream_widths(input_folding_model, max_bitwidth=max_bitwidth)

    # Insert DWCs where needed
    input_folding_model = insert_dwcs_if_needed(input_folding_model)
    input_folding_model.save(save_name)

    return input_folding_model


In [ ]:
'''
fc_layers = model.get_nodes_by_op_type("MVAU_hls")
# each tuple is (PE, SIMD, in_fifo_depth) for a layer
folding = [
    (16, 3, [128]),
    (32, 32, [128]),
    (16, 32, [128]),
    (16, 32, [128]),
    (4, 32, [81]),
    (1, 32, [2]),
    (1, 4, [2]),
    (1, 8, [128]),
    (5, 1, [3]),
]
for fcl, (pe, simd, ififodepth) in zip(fc_layers, folding):
    fcl_inst = getCustomOp(fcl)
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepths", ififodepth)

# use same SIMD values for the sliding window operators
swg_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator_rtl")
for i in range(len(swg_layers)):
    swg_inst = getCustomOp(swg_layers[i])
    simd = folding[i][1]
    swg_inst.set_nodeattr("SIMD", simd)

model = model.transform(GiveUniqueNodeNames())
model.save(set_onnx_checkpoint(Project_Info, "Folded Model"))
'''
model = folding_transform(model, set_onnx_checkpoint(Project_Info, "Folded Model"))
model = model.transform(GiveUniqueNodeNames())

In [ ]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild

def zynq_build_transform(input_zynq_model, save_name, brd_name):
    """
    Applies the ZynqBuild transformation to a model for the specified PYNQ board and clock period.

    Parameters:
        input_zynq_model (ModelWrapper): Folded model to transform.
        save_name (str): Directory to save the transformed model.
        brd_name (str): Name of the PYNQ board to target.
    
    Returns:
        ModelWrapper: The transformed model after applying ZynqBuild.
    """
    target_clk_ns = 10
    # Apply ZynqBuild transformation
    input_zynq_model = input_zynq_model.transform(ZynqBuild(platform=brd_name, period_ns=target_clk_ns))
    
    # Save the transformed model
    input_zynq_model.save(save_name)

    return input_zynq_model


In [ ]:
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver

def pynq_driver_transform(input_driver_model, save_name):
    """
    Applies the MakePYNQDriver transformation to the model to generate a PYNQ-compatible driver.

    Parameters:
        input_driver_model (ModelWrapper): ZynqBuild model to transform.
        save_name (str): Directory to save the transformed model.
    
    Returns:
        ModelWrapper: The transformed model with PYNQ driver compatibility.
    """
    # Apply MakePYNQDriver transformation
    input_driver_model = input_driver_model.transform(MakePYNQDriver("zynq-iodma"))
    
    # Save the transformed model
    input_driver_model.save(save_name)

    return input_driver_model


In [ ]:
model = zynq_build_transform(model, set_onnx_checkpoint(Project_Info, "Zynq Build"), Project_Info['Board_name'])
model = pynq_driver_transform(model, set_onnx_checkpoint(Project_Info, "Pynq Driver"))